In [1]:
!git clone https://github.com/thomasbrandon/mish-cuda.git

In [1]:
import shutil

shutil.rmtree("/app/_data/mish-cuda/build/")
!pip install /app/_data/mish-cuda
import mish_cuda

Processing /app/_data/mish-cuda
  Preparing metadata (setup.py) ... done
  Created wheel for mish-cuda: filename=mish_cuda-0.0.3-cp38-cp38-linux_x86_64.whl size=3420975 sha256=33334becae46034f2011616d1b4ff08db8780aa5e5703727d3bd5aaa00fcc998
  Stored in directory: /root/.cache/pip/wheels/62/4b/83/749668a9955ca03ad28e566d679417927169d50cb90dd6a2b6
Successfully built mish-cuda
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [1]:
import ast
import glob
import json
import os
import re
import shutil

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from IPython.core.interactiveshell import InteractiveShell
from PIL import Image
from sklearn.model_selection import KFold, StratifiedKFold
from tqdm import tqdm

InteractiveShell.ast_node_interactivity = "all"
from collections import Counter

import seaborn as sns
import torch
import torchvision
from IPython.display import clear_output
from torchvision.ops import box_iou

In [2]:
TRAIN_DF_PART = "/app/_data/tensorflow-great-barrier-reef/train.csv"
SEED = 38

In [3]:
df = pd.read_csv(TRAIN_DF_PART)
df["img_path"] = (
    "/app/_data/tensorflow-great-barrier-reef/train_images/video_"
    + df.video_id.astype("str")
    + "/"
    + df.video_frame.astype("str")
    + ".jpg"
)
df["annotations"] = df["annotations"].apply(lambda x: ast.literal_eval(x))
df["len_annotation"] = df["annotations"].str.len()
df["image_id"] = df["image_id"].str.replace("-", "_", regex=True)
df["new_img_path"] = "/app/_data/images/" + df["image_id"] + ".jpg"

# test

In [4]:
%cd /app/_data/SYOLOv4/ScaledYOLOv4

from custom_utils import get_model, load_image, plot_bboxes, y4predict
from models.experimental import attempt_load
from WBF.ensemble_boxes.ensemble_boxes_nms import nms
from WBF.ensemble_boxes.ensemble_boxes_wbf import weighted_boxes_fusion

from utils.datasets import letterbox
from utils.general import non_max_suppression

/app/_data/SYOLOv4/ScaledYOLOv4


In [5]:
MODEL_PATH = glob.glob('/app/_data/y4_weights_2/*.pt')
MODEL_PATH

['/app/_data/y4_weights_2/yolov4p7_val2_last_38.pt',
 '/app/_data/y4_weights_2/yolov4p7_val2_last_28.pt',
 '/app/_data/y4_weights_2/yolov4p7_val2_best.pt',
 '/app/_data/y4_weights_2/yolov4p7_val2_last_24.pt']

In [6]:
mod_path = MODEL_PATH[0]
IMG_SIZE = 2560
DEVICE = "cuda:0"

In [7]:
model, half, device = get_model(model_path=mod_path, device=DEVICE)

Fusing layers... Model Summary: 503 layers, 2.85956e+08 parameters, 2.7862e+08 gradients


# Test images

In [8]:
test = df.query('video_id == 2').reset_index(drop=True)

In [9]:
def tp_fp_fn(gt, prediction, conf_thr):
    ious = np.arange(0.3, 0.81, 0.05)
    TP, FP, FN = (
        np.zeros(ious.shape[0], "int16"),
        np.zeros(ious.shape[0], "int16"),
        np.zeros(ious.shape[0], "int16"),
    )
    if len(prediction):
        prediction = np.array(prediction)
        prediction = prediction[prediction[:, 4] >= conf_thr]
        bboxes = prediction[:, :4].astype("int")
        if bboxes.size != 0:
            if gt.size == 0:
                fp = bboxes.shape[0]
                FP = np.full(ious.shape[0], fp, "int16")
            else:
                iou_matrix = box_iou(torch.Tensor(gt), torch.Tensor(bboxes))
                for n, iou_thr in enumerate(ious):
                    x = torch.where(iou_matrix >= iou_thr)
                    tp = np.unique(x[0]).shape[0]
                    fp = bboxes.shape[0] - tp
                    fn = gt.shape[0] - tp
                    TP[n] = tp
                    FP[n] = fp
                    FN[n] = fn
    else:
        if gt.size != 0:
            fn = gt.shape[0]
            FN = np.full(ious.shape[0], fn, "int16")
    return TP, FP, FN

In [10]:
with open("/app/f2_results.json", "r") as f:
    res_dict = json.load(f)

In [11]:
def y4predict(img, h0, w0, dh, model, conf_thres, iou_thr, augment, method="nms"):
    h1 = img.shape[2] - dh
    w1 = img.shape[3]
    final_bboxes = []
    list_bboxes = []
    all_scores = []
    all_bboxes = []
    with torch.no_grad():
        prediction = model(img, augment=augment)[0].cpu().detach().numpy()[0]
    prediction = prediction[prediction[:, 4] > conf_thres]
    if prediction.size:
        conf = prediction[:, 4] * prediction[:, 5]
        bboxes_array = np.zeros(prediction[:, :4].shape)
        bboxes_array[:, 0] = (prediction[:, 0] - prediction[:, 2] / 2) / w1
        bboxes_array[:, 1] = (prediction[:, 1] - prediction[:, 3] / 2) / h1
        bboxes_array[:, 2] = (prediction[:, 0] + prediction[:, 2] / 2) / w1
        bboxes_array[:, 3] = (prediction[:, 1] + prediction[:, 3] / 2) / h1
        bboxes_array = np.clip(bboxes_array, 0, 1)
        all_bboxes.append(bboxes_array)
        all_scores.append(conf)
        classes = [["0"] * bboxes_array.shape[0]]
        if method == "nms":
            boxes, scores, _ = nms(
                boxes=all_bboxes,
                scores=all_scores,
                labels=classes,
                iou_thr=iou_thr,
                weights=None,
            )
        elif method == "soft_nms":
            boxes, scores, _ = soft_nms(
                boxes=all_bboxes,
                scores=all_scores,
                labels=classes,
                method=2,
                iou_thr=iou_thr,
                sigma=0.5,
                thresh=conf_thres,
                weights=None,
            )
        for b, s in zip(boxes, scores):
            xmin, ymin, xmax, ymax = b
            width = (xmax - xmin) * w0
            height = (ymax - ymin) * h0
            xmin *= w0
            ymin *= h0
            xmin, ymin, width, height = map(int, [xmin, ymin, width, height])
            list_bboxes.append([xmin, ymin, xmax*w0, ymax*h0, s])
            final_bboxes.append(
                " ".join(list(map(str, [s, xmin, ymin, width, height])))
            )
    return final_bboxes, list_bboxes

In [12]:
conf_thres = np.arange(0.1, 0.61, 0.01)
ious = np.arange(0.3, 0.81, 0.05)
res = np.zeros([conf_thres.shape[0], 3, ious.shape[0]])


# computing f2 score
for ix in tqdm(test.index.tolist()):
    img_path= test.loc[ix, 'img_path']
    img, h0, w0, dh = load_image(
        path = img_path,
        device = device,
        img=None,
        img_size=IMG_SIZE,
        half=half)
    
    _, list_bboxes = y4predict(img, h0, w0, dh, model, conf_thres=0.01, iou_thr = 0.4, augment=True, method="nms")
    gt = np.array([list(x.values()) for x in test.loc[ix, "annotations"]])
    if gt.size:
        gt[:, 2] = gt[:, 2] + gt[:, 0]
        gt[:, 3] = gt[:, 3] + gt[:, 1]
    for n, c_th in enumerate(conf_thres):
        TP, FP, FN = tp_fp_fn(gt, list_bboxes, c_th)
        res[n, 0, :] += TP
        res[n, 1, :] += FP
        res[n, 2, :] += FN

100% 8561/8561 [43:48<00:00,  3.26it/s]


In [13]:
F2 = np.zeros(conf_thres.shape[0])
for c in range(conf_thres.shape[0]):
    TP = res[c, 0, :]
    FP = res[c, 1, :]
    FN = res[c, 2, :]
    recall = TP / (TP + FN)
    precission = TP / (TP + FP)
    f2 = 5 * precission * recall / (4 * precission + recall + 1e-16)
    F2[c] = np.mean(f2)
if mod_path not in res_dict:
    res_dict[mod_path] = {
        IMG_SIZE: {
            "best": [
                np.round(conf_thres[np.argmax(F2)], 2),
                np.round(np.max(F2), 4),
            ],
            "all": list(np.round(F2, 4)),
        }
    }
else:
    res_dict[mod_path][IMG_SIZE] = {
        "best": [
            np.round(conf_thres[np.argmax(F2)], 2),
            np.round(np.max(F2), 4),
        ],
        "all": list(np.round(F2, 4)),
    }

In [14]:
mod_path
res_dict[mod_path]

'/app/_data/y4_weights_2/yolov4p7_val2_last_38.pt'

{2560: {'best': [0.27, 0.7453],
  'all': [0.719,
   0.7233,
   0.7267,
   0.7298,
   0.7336,
   0.7354,
   0.7365,
   0.7372,
   0.7389,
   0.7394,
   0.7406,
   0.7407,
   0.7407,
   0.7417,
   0.7427,
   0.7426,
   0.744,
   0.7453,
   0.745,
   0.7444,
   0.7437,
   0.7428,
   0.7428,
   0.7424,
   0.7409,
   0.7401,
   0.739,
   0.7372,
   0.7366,
   0.7362,
   0.7348,
   0.7337,
   0.7326,
   0.7303,
   0.7286,
   0.7276,
   0.7271,
   0.7251,
   0.7224,
   0.7203,
   0.7191,
   0.718,
   0.7156,
   0.7122,
   0.7093,
   0.7063,
   0.7018,
   0.7001,
   0.6974,
   0.6955,
   0.6933]}}

In [15]:
with open("/app/f2_results.json", "w") as f:
    json.dump(res_dict, f)